In [2]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("attention.pdf")
docs = loader.load()


In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
docs = text_splitter.split_documents(docs)

len(docs)

48

In [4]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

db=FAISS.from_documents(docs[:30],HuggingFaceEmbeddings())



c:\Users\Rushil Misra\Documents\projects\speak to pdf\.venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\Rushil Misra\Documents\projects\speak to pdf\.venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\Rushil Misra\Documents\projects\speak to pdf\.venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in ve

SIMILARITY SEARCH


In [5]:
query="An attention function can be described as mapping a query "
result=db.similarity_search(query)
result[0].page_content

'Scaled Dot-Product Attention\n Multi-Head Attention\nFigure 2: (left) Scaled Dot-Product Attention. (right) Multi-Head Attention consists of several\nattention layers running in parallel.\nof the values, where the weight assigned to each value is computed by a compatibility function of the\nquery with the corresponding key.\n3.2.1 Scaled Dot-Product Attention\nWe call our particular attention "Scaled Dot-Product Attention" (Figure 2). The input consists of\nqueries and keys of dimension dk, and values of dimension dv. We compute the dot products of the\nquery with all keys, divide each by√dk, and apply a softmax function to obtain the weights on the\nvalues.\nIn practice, we compute the attention function on a set of queries simultaneously, packed together\ninto a matrix Q. The keys and values are also packed together into matrices KandV. We compute\nthe matrix of outputs as:\nAttention( Q, K, V ) = softmax(QKT\n√dk)V (1)'

In [6]:
from langchain_community.llms import Ollama
llm=Ollama(model="llama2")


In [7]:
from langchain_core.prompts import ChatPromptTemplate


In [8]:
prompt =  ChatPromptTemplate.from_template('''
you are a simple assistant. answer the following question based only on the provided context
{context}
question : what is this document about?
answer :''')

In [9]:


from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)

In [10]:
document_chain.invoke({"context": docs})

"Based on the provided text, it appears that this document is about the application of a law or regulation related to the registration or voting process in American governments. The text mentions the passage of new laws since 2009 and the difficulty of the registration or voting process due to these laws. It also includes quotes from individuals expressing their opinions on the matter.\n\nThe document may be discussing various aspects of the law, such as its potential impact on voter turnout, the role of government in regulating the voting process, or the perspectives of different groups affected by the law. The attention mechanism used in the document's encoding self-attention layer 5 of 6 suggests that the model is able to capture long-distance dependencies and anaphora resolution, which could be useful for analyzing and understanding the complex relationships between different parts of the text."

In [11]:
retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000021CC1ED1ED0>)

In [12]:
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [13]:
response=retrieval_chain.invoke({"input":"what "})

In [14]:
response['answer']

'The document appears to be about a machine translation model that uses self-attention mechanisms, specifically scaled dot-product attention and multi-head attention, to attend to different parts of the input sequence simultaneously. The model is trained on a large dataset of sentence pairs and uses recurrent layers, such as long short-term memory (LSTM) and gated recurrent units (GRU), to process the input sequence. The document also mentions that the self-attention mechanism allows each position in the decoder to attend to all positions in the previous layer of the encoder, and that the model uses a scaling factor to prevent leftward information flow in the decoder. Additionally, the document notes that the representation dimensionality of the input sequence is typically larger than the number of output positions, and proposes restricting self-attention to a neighborhood of size r in the input sequence centered around the respective output position to improve computational performanc